In [3]:
import numpy as np
import tensorflow as tf
import pandas as pd

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
data_path = ""

data_dict = pd.read_csv(data_path).to_dict()

data = tf.data.Dataset.from_tensor_slices(data_dict)
data_shape = None

## Model

In [ ]:
model = Sequential([
    layers.Input(data_shape),
    layers.Dense(128, activation="relu", kernel_regularizer=None, kernel_initializer=None),
    layers.Dense(128, activation="relu", kernel_regularizer=None, kernel_initializer=None),
    layers.Dense(128, activation="relu", kernel_regularizer=None, kernel_initializer=None),
    layers.Dense(32, activation="relu", kernel_regularizer=None, kernel_initializer=None),
    layers.Dense(1, activation="softmax", kernel_regularizer=None, kernel_initializer=None)
])

model.compile()
model.summary()

ValueError: You must pass a `shape` argument.